In [ ]:
###########################################################################
#
#    covid19.py - Analyse data of the spread of the COVID19
#    Copyright (C) 2020 giovanni.organtini@uniroma1.it
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.
#
###########################################################################
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wget
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
urlc = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

# download data
w = pd.read_csv(wget.download(urlc), header = [0])

# select data
countries = ['Italy', 'US', 'Sweden']

for country in countries:
    select_country = w.loc[w['Country/Region'] == country]
    cases = select_country.values.tolist()[0][4:]

    plt.plot(cases, label = country)

# plot
plt.xlabel('Days since 22/1/2020')
plt.ylabel('Cases')
plt.legend()
plt.show()

plt.xlabel('Days since 22/1/2020')
plt.ylabel('Cases')
plt.yscale('log')
plt.legend()
plt.show()


# make a relative plot
for country in countries:
    select_country = w.loc[w['Country/Region'] == country]
    cases = select_country.values.tolist()[0][4:]
    cases = [c/max(cases) for c in cases]

    plt.plot(cases, label = country)

# plot
plt.xlabel('Days since 22/1/2020')
plt.ylabel('Relative cases')
plt.legend()
plt.show()

# make a plot of derivative
for country in countries:
    select_country = w.loc[w['Country/Region'] == country]
    cases = select_country.values.tolist()[0][4:]
    diff = []
    for x, y in zip(cases[0::], cases[1::]):
        diff.append(y - x)
    plt.plot(diff, label = country)

# plot
plt.xlabel('Days since 22/1/2020')
plt.ylabel('Daily cases')
plt.legend()
plt.show()
